In [1]:
!git clone https://github.com/google-research/timesfm.git

Cloning into 'timesfm'...
remote: Enumerating objects: 558, done.
remote: Counting objects: 100% (556/556), done.
remote: Compressing objects: 100% (284/284), done.
remote: Total 558 (delta 293), reused 468 (delta 243), pack-reused 2 (from 1)
Receiving objects: 100% (558/558), 1.65 MiB | 4.36 MiB/s, done.
Resolving deltas: 100% (293/293), done.


In [2]:
# Install required libraries
!pip install yfinance timesfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 35.0 MB/s et

In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=900edd8576522dad31f2dc521c43fcd36a67304dcff4d1eaf1d223b38c1cea42
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [3]:
import os
import time
import timesfm
from collections import defaultdict
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [4]:
# Load TimesFM model
timesfm_backend = "gpu"  # or 'cpu'
from jax._src import config
config.update("jax_platforms", {"cpu": "cpu", "gpu": "cuda", "tpu": ""}[timesfm_backend])

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

# انتخاب نماد سهام (مثال: AAPL برای اپل)
ticker = 'AAPL'

# بارگذاری داده‌های بورسی روزانه
data = yf.download(ticker, start='1900-01-01', end='2023-01-01', interval='1d')
data.reset_index(inplace=True)  # تبدیل شاخص به ستون

# تغییر نام ستون‌ها
data.rename(columns={'Date': 'date', 'Close': 'y'}, inplace=True)

# انتخاب ستون‌های مناسب
ts_cols = ["Open", "High", "Low", "y", "Volume"]

# محاسبه ویژگی‌های اضافی
data['MA_10'] = data['y'].rolling(window=10).mean()
data['MA_50'] = data['y'].rolling(window=50).mean()
data['RSI'] = 100 - (100 / (1 + data['y'].pct_change().rolling(window=14).apply(lambda x: (x[x > 0].mean() / -x[x < 0].mean()) if x[x < 0].mean() != 0 else np.inf)))

# حذف مقادیر NaN
data = data.dropna()

# انتخاب ستون‌های نهایی
final_cols = ["date", "Open", "High", "Low", "y", "Volume", "MA_10", "MA_50", "RSI"]
data = data[final_cols]

# ذخیره‌سازی داده‌ها در یک فایل CSV
data_path = 'stock_data.csv'
data.to_csv(data_path, index=False)

print(f"Number of records: {len(data)}")


[*********************100%***********************]  1 of 1 completed


Number of records: 10554


In [5]:
data

,date,Open,High,Low,y,Volume,MA_10,MA_50,RSI
49,1981-02-24,0.107143,0.107143,0.106027,0.106027,16979200,0.114676,0.132578,39.367995
50,1981-02-25,0.112723,0.113281,0.112723,0.112723,19488000,0.113783,0.132266,44.162188
51,1981-02-26,0.114397,0.114955,0.114397,0.114397,10841600,0.113448,0.132120,41.998503
52,1981-02-27,0.118304,0.119420,0.118304,0.118304,14761600,0.113616,0.132232,46.076704
53,1981-03-02,0.118862,0.119420,0.118862,0.118862,11760000,0.114118,0.132299,44.427514
...,...,...,...,...,...,...,...,...,...
10598,2022-12-23,130.919998,132.419998,129.639999,131.860001,63814900,136.839000,144.131800,47.618672
10599,2022-12-27,131.380005,131.410004,128.720001,130.029999,69007800,135.392999,143.964799,49.449664
10600,2022-12-28,129.669998,131.029999,125.870003,126.040001,85438400,133.449999,143.637399,46.803015
10601,2022-12-29,127.989998,130.479996,127.730003,129.610001,75703700,132.089999,143.354599,52.847661


In [6]:
import sys
import os

# اضافه کردن مسیر ماژول به sys.path
sys.path.append(os.path.join(os.getcwd(), 'timesfm/src/adapter'))

# حالا می‌توانید ماژول‌ها را وارد کنید
from utils import get_adapter_params, load_adapter_layer

In [7]:
import gc
import logging
import warnings
from datetime import datetime
from typing import Tuple

import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
from jax import numpy as jnp
from paxml import checkpoint_types, checkpoints, learners, tasks_lib, trainer_lib
from praxis import optimizers, pax_fiddle, py_utils, schedules
from tqdm import tqdm
from typing_extensions import Annotated
from timesfm import TimesFm, data_loader, patched_decoder

NestedMap = py_utils.NestedMap

warnings.filterwarnings("ignore")
cmdstanpy_logger = logging.getLogger("cmdstanpy")
absl_logger = logging.getLogger("absl")
cmdstanpy_logger.disabled = True
absl_logger.disabled = True

INPUT_PATCH_LEN = 32
OUTPUT_PATCH_LEN = 128
NUM_LAYERS = 20
MODEL_DIMS = 1280
QUANTILES = list(np.arange(1, 10) / 10.0)
EPS = 1e-7
RANDOM_SEED = 1234

def finetune(
    model_name: str = "google/timesfm-1.0-200m",
    checkpoint_path: str = None,
    datetime_col: str = "date",
    ts_cols: list[str] = [],
    normalize: bool = True,
    num_cov_cols: int = None,
    context_len: int = 64,
    horizon_len: int = 1,
    freq: str = "D",
    data_path: str = "timeseries_data.csv",
    boundaries: Tuple[int, int, int] = (0, 0, 0),
    backend: str = "gpu",
    batch_size: int = 16,
    num_epochs: int = 10,
    learning_rate: float = 0.001,
    adam_epsilon: float = 1e-8,
    adam_clip_threshold: float = 1.0,
    cos_initial_decay_value: float = 0.001,
    cos_final_decay_value: float = 0.0001,
    cos_decay_steps: int = 1000,
    ema_decay: float = 0.9999,
    early_stop_patience: int = 5,
    use_lora: bool = False,
    lora_rank: int = 8,
    lora_target_modules: str = "all",
    use_dora: bool = False,
    use_linear_probing: bool = False,
    checkpoint_dir: str = "./checkpoints",
) -> None:
    key = jax.random.PRNGKey(seed=RANDOM_SEED)

    data_df = pd.read_csv(open(data_path, "r"))

    if boundaries == (0, 0, 0):
        boundaries = [
            int(len(data_df) * 0.85),
            int(len(data_df) * 0.95),
            len(data_df) - 1,
        ]

    ts_cols = [col for col in data_df.columns if col != datetime_col]

    dtl = data_loader.TimeSeriesdata(
        data_path=data_path,
        datetime_col=datetime_col,
        num_cov_cols=num_cov_cols,
        cat_cov_cols=None,
        ts_cols=np.array(ts_cols),
        train_range=[0, boundaries[0]],
        val_range=[boundaries[0], boundaries[1]],
        test_range=[boundaries[1], boundaries[2]],
        hist_len=context_len,
        pred_len=horizon_len,
        batch_size=batch_size,
        freq=freq,
        normalize=normalize,
        epoch_len=None,
        holiday=False,
        permute=False,
    )

    train_batches = dtl.tf_dataset(mode="train", shift=1).batch(batch_size)
    val_batches = dtl.tf_dataset(mode="val", shift=horizon_len)

    for tbatch in tqdm(train_batches.as_numpy_iterator()):
        pass
    print(tbatch[0].shape)

    tfm = TimesFm(
        context_len=context_len,
        horizon_len=horizon_len,
        input_patch_len=INPUT_PATCH_LEN,
        output_patch_len=OUTPUT_PATCH_LEN,
        num_layers=NUM_LAYERS,
        model_dims=MODEL_DIMS,
        backend=backend,
        per_core_batch_size=batch_size,
        quantiles=QUANTILES,
    )

    if checkpoint_path:
        tfm.load_from_checkpoint(
            checkpoint_path=checkpoint_path,
            checkpoint_type=checkpoints.CheckpointType.FLAX,
        )
    else:
        tfm.load_from_checkpoint(
            repo_id=model_name,
            checkpoint_type=checkpoints.CheckpointType.FLAX,
        )

    model = pax_fiddle.Config(
        patched_decoder.PatchedDecoderFinetuneModel,
        name="patched_decoder_finetune",
        core_layer_tpl=tfm.model_p,
    )

    if use_lora:
        load_adapter_layer(
            mdl_vars=tfm._train_state.mdl_vars,
            model=model.core_layer_tpl,
            lora_rank=lora_rank,
            lora_target_modules=lora_target_modules,
            use_dora=use_dora,
        )

    @pax_fiddle.auto_config
    def build_learner() -> learners.Learner:
        bprop_variable_inclusion = []
        bprop_variable_exclusion = []
        if use_lora:
            bprop_variable_inclusion.append(r"^.*lora.*$")
            if use_dora:
                bprop_variable_inclusion.append(r"^.*dora.*$")
        elif use_linear_probing:
            bprop_variable_exclusion = [".*/stacked_transformer_layer/.*"]

        return pax_fiddle.Config(
            learners.Learner,
            name="learner",
            loss_name="avg_qloss",
            optimizer=optimizers.Adam(
                epsilon=adam_epsilon,
                clip_threshold=adam_clip_threshold,
                learning_rate=learning_rate,
                lr_schedule=pax_fiddle.Config(
                    schedules.Cosine,
                    initial_value=cos_initial_decay_value,
                    final_value=cos_final_decay_value,
                    total_steps=cos_decay_steps,
                ),
                ema_decay=ema_decay,
            ),
            bprop_variable_exclusion=bprop_variable_exclusion,
            bprop_variable_inclusion=bprop_variable_inclusion,
        )

    task_p = tasks_lib.SingleTask(
        name="ts-learn",
        model=model,
        train=tasks_lib.SingleTask.Train(
            learner=build_learner(),
        ),
    )

    task_p.model.ici_mesh_shape = [1, 1, 1]
    task_p.model.mesh_axis_names = ["replica", "data", "mdl"]

    DEVICES = np.array(jax.devices()).reshape([1, 1, 1])
    jax.sharding.Mesh(DEVICES, ["replica", "data", "mdl"])

    num_devices = jax.local_device_count()
    print(f"num_devices: {num_devices}")
    print(f"device kind: {jax.local_devices()[0].device_kind}")

    jax_task = task_p
    key, init_key = jax.random.split(key)

    def process_train_batch(batch):

        past_ts = batch[0].reshape( batch[0].shape[0] * len(ts_cols), -1)
        actual_ts = batch[3].reshape(batch[3].shape[0] * len(ts_cols), -1)
        return NestedMap(input_ts=past_ts, actual_ts=actual_ts)

    def process_eval_batch(batch):
        past_ts = batch[0]
        actual_ts = batch[3]
        return NestedMap(input_ts=past_ts, actual_ts=actual_ts)

    jax_model_states, _ = trainer_lib.initialize_model_state(
        jax_task,
        init_key,
        process_train_batch(tbatch),
        checkpoint_type=checkpoint_types.CheckpointType.GDA,
    )
    jax_model_states.mdl_vars["params"]["core_layer"] = tfm._train_state.mdl_vars[
        "params"
    ]
    gc.collect()

    jax_task = task_p

    def train_step(states, prng_key, inputs):
        return trainer_lib.train_step_single_learner(jax_task, states, prng_key, inputs)

    def eval_step(states, prng_key, inputs):
        states = states.to_eval_state()
        return trainer_lib.eval_step_single_learner(jax_task, states, prng_key, inputs)

    key, train_key, eval_key = jax.random.split(key, 3)
    train_prng_seed = jax.random.split(train_key, num=jax.local_device_count())
    eval_prng_seed = jax.random.split(eval_key, num=jax.local_device_count())

    p_train_step = jax.pmap(train_step, axis_name="batch")
    p_eval_step = jax.pmap(eval_step, axis_name="batch")

    replicated_jax_states = trainer_lib.replicate_model_state(jax_model_states)

    def reshape_batch_for_pmap(batch, num_devices):
        def _reshape(input_tensor):
            bsize = input_tensor.shape[0]
            residual_shape = list(input_tensor.shape[1:])
            nbsize = bsize // num_devices
            return jnp.reshape(input_tensor, [num_devices, nbsize] + residual_shape)

        return jax.tree.map(_reshape, batch)

    patience = 0
    best_eval_loss = 1e7
    checkpoint_dir = f"{checkpoint_dir}/run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    for epoch in range(num_epochs):
        if patience >= early_stop_patience:
            print("Early stopping.")
            break
        print(f"Epoch: {epoch + 1}")
        train_its = train_batches.as_numpy_iterator()
        train_losses = []
        for batch in tqdm(train_its):
            tbatch = process_train_batch(batch)
            tbatch = reshape_batch_for_pmap(tbatch, num_devices)
            replicated_jax_states, step_fun_out = p_train_step(
                replicated_jax_states, train_prng_seed, tbatch
            )
            train_losses.append(step_fun_out.loss[0])

        avg_train_loss = np.mean(train_losses)

        print("Starting eval.")
        val_its = val_batches.as_numpy_iterator()
        eval_losses = []
        for ev_batch in tqdm(val_its):
            ebatch = process_eval_batch(ev_batch)
            ebatch = reshape_batch_for_pmap(ebatch, num_devices)
            _, step_fun_out = p_eval_step(replicated_jax_states, eval_prng_seed, ebatch)
            eval_losses.append(step_fun_out.loss[0])

        avg_eval_loss = np.mean(eval_losses)

        print(f"Train Loss: {avg_train_loss}, Val Loss: {avg_eval_loss}")

        if avg_eval_loss < best_eval_loss or np.isnan(avg_eval_loss):
            best_eval_loss = avg_eval_loss
            print("Saving checkpoint.")
            jax_state_for_saving = py_utils.maybe_unreplicate_for_fully_replicated(
                replicated_jax_states
            )
            if use_lora:
                adapter_params = get_adapter_params(
                    params=jax_state_for_saving.mdl_vars,
                    lora_target_modules=lora_target_modules,
                    num_layers=NUM_LAYERS,
                    use_dora=use_dora,
                )
                jax_state_for_saving.mdl_vars["params"] = adapter_params

            checkpoints.save_checkpoint(
                jax_state_for_saving, checkpoint_dir, overwrite=True
            )

            patience = 0
            del jax_state_for_saving
            gc.collect()
        else:
            patience += 1
            print(f"patience: {patience}")
    print("Fine-tuning completed.")


In [9]:
ts_cols = [col for col in data.columns if col != "date"]

finetune(
    model_name="google/timesfm-1.0-200m",  # یا هر مدل دیگر
    checkpoint_path=None,  # اگر از چک‌پوینت خاصی می‌خواهید استفاده کنید، مسیر را مشخص کنید
    datetime_col="date",
    ts_cols=ts_cols,
    num_cov_cols=None,  # تعداد ستون‌های کوواریانس عددی
    context_len=64,  # طول پنجره زمینه
    horizon_len=1,  # طول پیش‌بینی
    freq="D",  # فرکانس داده‌ها (روزانه)
    data_path="stock_data.csv",  # مسیر فایل CSV
    backend="gpu",  # یا 'cpu' یا 'tpu'
    batch_size=len(ts_cols),
    num_epochs=20,
    use_lora=True,
    learning_rate=0.001,
    adam_epsilon=1e-8,
    adam_clip_threshold=1.0,
    cos_initial_decay_value=0.001,
    cos_final_decay_value=0.0001,
    cos_decay_steps=1000,
    ema_decay=0.9999,
    early_stop_patience=5,
    checkpoint_dir="./checkpoints",

)



2024-09-06 16:48:26.269971: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.6.68). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
2024-09-06 16:48:26.813504: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-09-06 16:48:26.813749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-09-06 16:48:26.813901: W tensorflow/stream_executor/platform/defa

(2, 8, 64)
Multiprocessing context has already been set.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.73k [00:00<?, ?B/s]

...

2227it [04:06,  9.04it/s]


Starting eval.


1056it [00:26, 39.35it/s]


Train Loss: 0.6121160984039307, Val Loss: 0.8136546015739441
patience: 3
Epoch: 17


KeyboardInterrupt: 

In [8]:
from timesfm import TimesFm, freq_map, data_loader

from tqdm import tqdm
import numpy as np
import pandas as pd


tfm = TimesFm(
    context_len=64,
    horizon_len=128,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

2024-09-06 18:12:31.993216: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.6.68). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.
Constructed model weights in 4.32 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.
Restored checkpoint in 5.20 seconds.
Jitting decoding.
Jitted decoding in 24.65 seconds.


In [9]:
import sys
import os

# اضافه کردن مسیر ماژول به sys.path
sys.path.append(os.path.join(os.getcwd(), 'timesfm/src/adapter'))

# حالا می‌توانید ماژول‌ها را وارد کنید
from utils import load_adapter_checkpoint, load_adapter_layer

In [ ]:
!gdown --folder "https://drive.google.com/drive/folders/1-0PTQRS7AIlKgOEwcud4PKcJzWYp0i8c?usp=sharing"

Retrieving folder contents
Retrieving folder 1-60ou0L_1teRZXrNbcPCEnLl40Zx_8Sa run_20240816_084744
Retrieving folder 1-LsrhhNTXMFlcmPS445Ss8DADpfZUH2q checkpoint_2089.orbax-checkpoint-tmp-1723798443087197
Retrieving folder 10DxPWvx9Np3TazLz7M62rC5TniZWrTsv metadata
Processing file 10sdFtbDxIStMa29YADwlLC1c0tPeL1ck metadata
Retrieving folder 104qKSSZCn-msslTCwGsFjGnWbpoxrrms state
Retrieving folder 1-D-v3_TdwslriaMV3HhrvI8gu27YKFLE run_20240816_085627
Retrieving folder 1-bhCuSZpwS30D-rKfAE9Jx0MbN6l4EZO checkpoint_2089.orbax-checkpoint-tmp-1723798968067947
Retrieving folder 10XJKrfd4w6Bcy9V4E5dMeKVrqgOsGHSX metadata
Processing file 10cTrhC6AQ0v5nLxpb7o60uYDjjL79y-1 metadata
Retrieving folder 10WpYdvISOCzBwSGdebEZ011C93K-iW6F state
Retrieving folder 1-2UxA4R93tyNtjqw_rb1cZ8fM0VJSIQ6 run_20240816_090802
Retrieving folder 1-HGw6QC0uZ5Srdit83SFIpFul379nI92 checkpoint_12534.orbax-checkpoint-tmp-1723800729967276
Retrieving folder 104l804tra4Duhw-J50vuJw1hT1cXoNEE metadata
Processing file 10row

In [10]:
load_adapter_checkpoint(
    model=tfm,
    adapter_checkpoint_path="/content/checkpoints/run_20240906_164928",
    lora_rank=8,
    lora_target_modules="all",
    use_dora=False,

)

Restoring adapter checkpoint from /content/checkpoints/run_20240906_164928.
Restored adapter checkpoint in 5.16 seconds.
Jitting decoding.
Jitted decoding in 19.17 seconds.


In [11]:
import yfinance as yf
# Download stock data from Yahoo Finance
ticker = 'AAPL'  # Example: Apple Inc.
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")

data = data.dropna()


df = data[['Close']].reset_index()
df.columns = ['date', 'close']
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df

[*********************100%***********************]  1 of 1 completed


,close
date,
2023-01-03,125.070000
2023-01-04,126.360001
2023-01-05,125.019997
2023-01-06,129.619995
2023-01-09,130.149994
...,...
2023-12-22,193.600006
2023-12-26,193.050003
2023-12-27,193.149994


In [12]:


def get_batched_data_fn(batch_size: int = 32, context_len: int = 32, horizon_len: int = 1):
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(df) - (context_len + horizon_len), horizon_len):
        num_examples += 1
        context_end = start + context_len
        examples["inputs"].append(df["close"][start:context_end].tolist())
        examples["outputs"].append(df["close"][context_end:context_end + horizon_len].tolist())
        examples["dates"].append(df.index[start:context_end + horizon_len].tolist())  # Add dates to examples

    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size): ((i + 1) * batch_size)] for k, v in examples.items()}

    return data_fn

In [13]:
import timesfm

context_len = 64
batch_size = 1  # ثابت نگه‌داشتن batch size
horizon_len = 1  # ثابت نگه‌داشتن horizon length



In [14]:
from collections import defaultdict

In [15]:
      input_data = get_batched_data_fn(batch_size=batch_size, context_len=context_len, horizon_len=horizon_len)

      results = []

      # Process in smaller sub-batches
      for i, example in enumerate(input_data()):
          raw_forecast, _ = tfm.forecast(
              inputs=example["inputs"], freq=[0] * len(example["inputs"])
          )

          for j in range(len(example["inputs"])):
              result = {
                  'input_index': j,
                  'train_start_date': str(example["dates"][j][0]),
                  'train_end_date': str(example["dates"][j][-2]),
                  'predict_date': str(example["dates"][j][-1]),
                  'raw_forecast': raw_forecast[j][0],
                  'y_actual': example["outputs"][j][0],
                  'batch_size': batch_size,
                  'context_len': context_len,

              }
              results.append(result)

In [16]:
 results_df = pd.DataFrame(results)
 results_df

,input_index,train_start_date,train_end_date,predict_date,raw_forecast,y_actual,batch_size,context_len
0,0,2023-01-03 00:00:00,2023-04-04 00:00:00,2023-04-05 00:00:00,165.545746,163.759995,1,64
1,0,2023-01-04 00:00:00,2023-04-05 00:00:00,2023-04-06 00:00:00,163.436020,164.660004,1,64
2,0,2023-01-05 00:00:00,2023-04-06 00:00:00,2023-04-10 00:00:00,163.708618,162.029999,1,64
3,0,2023-01-06 00:00:00,2023-04-10 00:00:00,2023-04-11 00:00:00,160.743683,160.800003,1,64
4,0,2023-01-09 00:00:00,2023-04-11 00:00:00,2023-04-12 00:00:00,159.515839,160.100006,1,64
...,...,...,...,...,...,...,...,...
180,0,2023-09-21 00:00:00,2023-12-20 00:00:00,2023-12-21 00:00:00,194.651886,194.679993,1,64
181,0,2023-09-22 00:00:00,2023-12-21 00:00:00,2023-12-22 00:00:00,194.010590,193.600006,1,64
182,0,2023-09-25 00:00:00,2023-12-22 00:00:00,2023-12-26 00:00:00,192.996643,193.050003,1,64
183,0,2023-09-26 00:00:00,2023-12-26 00:00:00,2023-12-27 00:00:00,192.331039,193.149994,1,64


In [17]:
import pandas as pd
from sklearn.metrics import mean_squared_error, accuracy_score

def calculate_mse(context_results_df):
    return mean_squared_error(context_results_df['y_actual'], context_results_df['raw_forecast'])

def calculate_accuracy(context_results_df):
    correct_predictions = 0
    total_predictions = 0
    for i in range(1, len(context_results_df)):
        actual_today = context_results_df['y_actual'].iloc[i]
        predicted_today = context_results_df['raw_forecast'].iloc[i]
        actual_yesterday = context_results_df['y_actual'].iloc[i - 1]
        actual_movement = actual_today - actual_yesterday
        predicted_movement = predicted_today - actual_yesterday
        if (actual_movement > 0 and predicted_movement > 0) or (actual_movement < 0 and predicted_movement < 0):
            correct_predictions += 1
        total_predictions += 1

    return correct_predictions / total_predictions if total_predictions > 0 else 0

def calculate_profit(context_results_df, initial_capital=100):
    capital = initial_capital
    position = 0

    for i in range(1, len(context_results_df)):
        today_price = context_results_df['y_actual'].iloc[i]
        yesterday_price = context_results_df['y_actual'].iloc[i - 1]
        predicted_today_price = context_results_df['raw_forecast'].iloc[i]

        if predicted_today_price > yesterday_price:
            if position == 0:
                position = capital / yesterday_price
                capital = 0
        elif predicted_today_price < yesterday_price:
            if position > 0:
                capital = position * yesterday_price
                position = 0

    if position > 0:
        capital = position * context_results_df.iloc[-1]['y_actual']

    return capital - initial_capital

def calculate_actual_direction_accuracy(context_results_df):
    def get_direction(current_price, next_price):
        return 1 if next_price > current_price else 0

    real_directions = []
    predicted_directions = []

    for i in range(1, len(context_results_df)):
        real_directions.append(get_direction(context_results_df['y_actual'].iloc[i - 1], context_results_df['y_actual'].iloc[i]))
        predicted_directions.append(get_direction(context_results_df['y_actual'].iloc[i - 1], context_results_df['raw_forecast'].iloc[i]))

    return accuracy_score(real_directions, predicted_directions)

def calculate_direction_accuracy(context_results_df):
    def get_direction(current_price, next_price):
        return 1 if next_price > current_price else 0

    real_directions = []
    predicted_directions = []

    for i in range(1, len(context_results_df)):
        real_directions.append(get_direction(context_results_df['y_actual'].iloc[i - 1], context_results_df['y_actual'].iloc[i]))
        predicted_directions.append(get_direction(context_results_df['raw_forecast'].iloc[i - 1], context_results_df['raw_forecast'].iloc[i]))

    return accuracy_score(real_directions, predicted_directions)

def calculate_direction_profit(context_results_df, initial_capital=100):
    capital = initial_capital
    position = 0

    for i in range(2, len(context_results_df)):
        yesterday_price = context_results_df['y_actual'].iloc[i - 1]
        yesterday_predicted_today_price = context_results_df['raw_forecast'].iloc[i - 1]
        predicted_today_price = context_results_df['raw_forecast'].iloc[i]

        if predicted_today_price > yesterday_predicted_today_price:
            if position == 0:
                position = capital / yesterday_price
                capital = 0
        elif predicted_today_price < yesterday_predicted_today_price:
            if position > 0:
                capital = position * yesterday_price
                position = 0

    if position > 0:
        capital = position * context_results_df.iloc[-1]['y_actual']

    return capital - initial_capital

def calculate_results_for_context_len(context_len, results_by_context_len):
    # فیلتر کردن داده‌ها برای context_len مشخص شده
    context_results_df =results_by_context_len

    mse_value = calculate_mse(context_results_df)
    accuracy_value = calculate_accuracy(context_results_df)
    profit = calculate_profit(context_results_df)
    direction_accuracy = calculate_direction_accuracy(context_results_df)
    actual_direction_accuracy = calculate_actual_direction_accuracy(context_results_df)
    direction_profit = calculate_direction_profit(context_results_df)

    # ساخت دیکشنری از نتایج محاسبه شده
    result = {
        'context_len': context_len,
        'mse': mse_value,
        'accuracy': accuracy_value,
        'direction_accuracy': direction_accuracy,
        'actual_direction_accuracy': actual_direction_accuracy,
        'direction_profit': direction_profit,
        'profit': profit
    }

    return pd.DataFrame([result])

# فراخوانی تابع برای یک context_len خاص
context_len = 64  # برای مثال
final_result_df = calculate_results_for_context_len(context_len, results_df)
print(final_result_df)


   context_len       mse  accuracy  direction_accuracy  \
0           64  4.828243   0.48913            0.505435   

   actual_direction_accuracy  direction_profit     profit  
0                    0.48913         10.448894  10.377625  


In [18]:
initial_investment = 100
initial_price = results_df['y_actual'].iloc[0]
final_price = results_df['y_actual'].iloc[-1]


# Calculate the number of shares bought initially
shares_bought = initial_investment / initial_price

# Calculate the final value of the investment
final_investment_value = shares_bought * final_price

# Calculate the total profit
buy_and_hold_profit = final_investment_value - initial_investment
buy_and_hold_profit

18.209580071141218